# Infer-5-Skills-IRT : Evaluation de Competences et Psychometrie

**Serie** : Programmation Probabiliste avec Infer.NET (5/12)  
**Duree estimee** : 60 minutes  
**Prerequis** : Infer-4-Bayesian-Networks

---

## Objectifs

- Comprendre les modeles d'evaluation cognitive
- Implementer le modele IRT (Item Response Theory) Difficulty-Ability
- Construire le modele DINA (Noisy-And) pour competences multiples
- Gerer les relations many-to-many entre competences et questions
- Estimer les parametres slip et guess

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-4-Bayesian-Networks](Infer-4-Bayesian-Networks.ipynb) | [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb) |

---

## 1. Configuration

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

## 2. Introduction a l'Evaluation Cognitive

### Probleme

Comment evaluer les competences d'un etudiant a partir de ses reponses a un test ?

### Defis

| Defi | Description |
|------|-------------|
| **Capacite latente** | La competence n'est pas directement observable |
| **Bruit** | Les reponses peuvent etre correctes par chance ou incorrectes par erreur |
| **Difficulte variable** | Les questions ont des difficultes differentes |
| **Competences multiples** | Une question peut necessiter plusieurs competences |

### Approches

| Modele | Caracteristique |
|--------|----------------|
| **IRT classique** | Capacite unidimensionnelle + difficulte des questions |
| **DINA** | Competences discretes multiples + matrice Q |
| **Bayesien hierarchique** | Parametres appris de maniere adaptative |

## 3. Modele IRT : Difficulty-Ability

### Formulation

$$P(\text{correct}_{ij}) = \sigma(\text{capacite}_i - \text{difficulte}_j)$$

Ou $\sigma$ est la fonction logistique (ou probit dans notre cas).

### Structure

```
capacite[i] ~ N(0, 1)    pour chaque etudiant
difficulte[j] ~ N(0, 1)  pour chaque question

avantage[i,j] = capacite[i] - difficulte[j]
reponse[i,j] ~ Probit(avantage[i,j])
```

In [ ]:
// Modele IRT Difficulty-Ability

int nEtudiants = 10;
int nQuestions = 5;

// Donnees simulees : matrice de reponses (true = correct)
bool[,] reponses = new bool[,] {
    // Q1    Q2     Q3     Q4     Q5
    { true,  true,  true,  false, false },  // Etudiant 1 (bon)
    { true,  true,  false, false, false },  // Etudiant 2
    { true,  false, false, false, false },  // Etudiant 3 (faible)
    { true,  true,  true,  true,  false },  // Etudiant 4 (tres bon)
    { false, false, false, false, false },  // Etudiant 5 (tres faible)
    { true,  true,  false, true,  false },  // Etudiant 6
    { true,  true,  true,  false, true },   // Etudiant 7
    { true,  false, true,  false, false },  // Etudiant 8
    { true,  true,  true,  true,  true },   // Etudiant 9 (excellent)
    { false, true,  false, false, false }   // Etudiant 10
};

// Definition du modele
Range etudiant = new Range(nEtudiants).Named("etudiant");
Range question = new Range(nQuestions).Named("question");

// Capacites latentes des etudiants
VariableArray<double> capacite = Variable.Array<double>(etudiant).Named("capacite");
capacite[etudiant] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(etudiant);

// Difficultes des questions
VariableArray<double> difficulte = Variable.Array<double>(question).Named("difficulte");
difficulte[question] = Variable.GaussianFromMeanAndPrecision(0, 1).ForEach(question);

// Discrimination (bruit)
Variable<double> discrimination = Variable.GammaFromShapeAndScale(2, 0.5).Named("discrimination");

// Reponses observees
VariableArray2D<bool> reponseVar = Variable.Array<bool>(etudiant, question).Named("reponse");

using (Variable.ForEach(etudiant))
{
    using (Variable.ForEach(question))
    {
        Variable<double> avantage = capacite[etudiant] - difficulte[question];
        Variable<double> avantageBruite = Variable.GaussianFromMeanAndPrecision(avantage, discrimination);
        reponseVar[etudiant, question] = (avantageBruite > 0);
    }
}

// Observations
reponseVar.ObservedValue = reponses;

Console.WriteLine("Modele IRT defini.");

In [ ]:
// Inference
InferenceEngine moteurIRT = new InferenceEngine(new ExpectationPropagation());
moteurIRT.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian[] capacitePost = moteurIRT.Infer<Gaussian[]>(capacite);
Gaussian[] difficultePost = moteurIRT.Infer<Gaussian[]>(difficulte);

Console.WriteLine("=== Capacites des etudiants (IRT) ===");
for (int i = 0; i < nEtudiants; i++)
{
    int nCorrect = 0;
    for (int j = 0; j < nQuestions; j++) if (reponses[i, j]) nCorrect++;
    Console.WriteLine($"Etudiant {i+1} : capacite = {capacitePost[i].GetMean():F2} +/- {Math.Sqrt(capacitePost[i].GetVariance()):F2} (score: {nCorrect}/{nQuestions})");
}

Console.WriteLine("\n=== Difficultes des questions ===");
for (int j = 0; j < nQuestions; j++)
{
    int nReussi = 0;
    for (int i = 0; i < nEtudiants; i++) if (reponses[i, j]) nReussi++;
    Console.WriteLine($"Question {j+1} : difficulte = {difficultePost[j].GetMean():F2} (taux reussite: {nReussi}/{nEtudiants})");
}

### Analyse

- Les etudiants avec plus de bonnes reponses ont une capacite estimee plus elevee
- Les questions reussies par peu d'etudiants ont une difficulte elevee
- L'incertitude est plus grande pour les etudiants aux performances moyennes

## 4. Modele DINA : Competences Discretes

### Motivation

Le modele IRT suppose une capacite **unidimensionnelle**. En realite, un test peut evaluer **plusieurs competences**.

### Modele DINA (Deterministic Input, Noisy And)

- Chaque etudiant possede ou non chaque competence (variable binaire)
- Chaque question necessite un sous-ensemble de competences (matrice Q)
- Reponse correcte si **toutes** les competences requises sont presentes
- Avec des erreurs slip et guess

### Parametres

| Parametre | Description |
|-----------|-------------|
| **Slip** | P(incorrect \| a toutes les competences) - erreur d'inattention |
| **Guess** | P(correct \| manque une competence) - reponse au hasard |

In [ ]:
// Modele DINA simplifie

int nEtud = 8;
int nQuest = 6;
int nCompetences = 3;

// Matrice Q : quelles competences sont requises pour chaque question
// Q1 : C1 seulement
// Q2 : C2 seulement
// Q3 : C3 seulement
// Q4 : C1 et C2
// Q5 : C2 et C3
// Q6 : C1, C2 et C3
bool[,] matriceQ = new bool[,] {
    // C1    C2     C3
    { true,  false, false },  // Q1
    { false, true,  false },  // Q2
    { false, false, true  },  // Q3
    { true,  true,  false },  // Q4
    { false, true,  true  },  // Q5
    { true,  true,  true  }   // Q6
};

// Donnees : reponses des etudiants
bool[,] repDINA = new bool[,] {
    // Q1    Q2     Q3     Q4     Q5     Q6
    { true,  true,  true,  true,  true,  true  },  // E1 : a tout
    { true,  true,  false, true,  false, false },  // E2 : C1, C2 seulement
    { true,  false, true,  false, false, false },  // E3 : C1, C3 seulement
    { false, true,  true,  false, true,  false },  // E4 : C2, C3 seulement
    { true,  false, false, false, false, false },  // E5 : C1 seulement
    { false, true,  false, false, false, false },  // E6 : C2 seulement
    { false, false, true,  false, false, false },  // E7 : C3 seulement
    { false, false, false, false, false, false }   // E8 : rien
};

Console.WriteLine("Donnees DINA definies.");
Console.WriteLine("\nMatrice Q (competences requises par question) :");
for (int q = 0; q < nQuest; q++)
{
    string req = "";
    for (int c = 0; c < nCompetences; c++)
        if (matriceQ[q, c]) req += $"C{c+1} ";
    Console.WriteLine($"  Q{q+1} : {req}");
}

In [ ]:
// Implementation du modele DINA

Range rEtud = new Range(nEtud).Named("etudiant");
Range rQuest = new Range(nQuest).Named("question");
Range rComp = new Range(nCompetences).Named("competence");

// Prior sur les competences (50% d'avoir chaque competence)
Variable<double> priorCompetence = Variable.Beta(1, 1).Named("priorComp");

// Competences des etudiants
VariableArray2D<bool> aCompetence = Variable.Array<bool>(rEtud, rComp).Named("aCompetence");
aCompetence[rEtud, rComp] = Variable.Bernoulli(priorCompetence).ForEach(rEtud, rComp);

// Parametres slip et guess
Variable<double> slip = Variable.Beta(1, 9).Named("slip");   // Prior : faible (~0.1)
Variable<double> guess = Variable.Beta(1, 9).Named("guess"); // Prior : faible (~0.1)

// Matrice Q comme constante
VariableArray2D<bool> qMatrix = Variable.Observed(matriceQ, rQuest, rComp).Named("qMatrix");

// Reponses
VariableArray2D<bool> reponseDINA = Variable.Array<bool>(rEtud, rQuest).Named("reponse");

using (Variable.ForEach(rEtud))
{
    using (Variable.ForEach(rQuest))
    {
        // Calculer si l'etudiant a toutes les competences requises pour cette question
        // C'est un AND sur toutes les competences requises
        Variable<bool> toutesCompetences = Variable.Bernoulli(1.0);  // Commence a true
        
        using (Variable.ForEach(rComp))
        {
            using (Variable.If(qMatrix[rQuest, rComp]))  // Si cette competence est requise
            {
                // L'etudiant doit l'avoir
                toutesCompetences.SetTo(toutesCompetences & aCompetence[rEtud, rComp]);
            }
        }
        
        // Si a toutes les competences : reussite sauf slip
        // Sinon : echec sauf guess
        using (Variable.If(toutesCompetences))
        {
            reponseDINA[rEtud, rQuest] = !Variable.Bernoulli(slip);  // 1 - slip
        }
        using (Variable.IfNot(toutesCompetences))
        {
            reponseDINA[rEtud, rQuest] = Variable.Bernoulli(guess);  // guess
        }
    }
}

Console.WriteLine("Modele DINA defini (complexe).");

> **Note** : Le modele DINA complet avec Variable.ForEach et AND dynamique est complexe dans Infer.NET. Nous allons utiliser une version simplifiee ci-dessous.

In [ ]:
// Version simplifiee : modele DINA pour une question specifique

// Question Q4 necessite C1 ET C2
Variable<bool> aC1 = Variable.Bernoulli(0.5).Named("aC1");
Variable<bool> aC2 = Variable.Bernoulli(0.5).Named("aC2");

Variable<double> slipQ4 = Variable.Beta(1, 9);
Variable<double> guessQ4 = Variable.Beta(1, 9);

Variable<bool> toutesCompQ4 = aC1 & aC2;
Variable<bool> reponseQ4 = Variable.New<bool>();

using (Variable.If(toutesCompQ4))
{
    reponseQ4.SetTo(!Variable.Bernoulli(slipQ4));
}
using (Variable.IfNot(toutesCompQ4))
{
    reponseQ4.SetTo(Variable.Bernoulli(guessQ4));
}

// Observation : l'etudiant a repondu correctement
reponseQ4.ObservedValue = true;

InferenceEngine mDINA = new InferenceEngine();
mDINA.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== DINA : Inference des competences ===");
Console.WriteLine($"Question Q4 necessite C1 ET C2");
Console.WriteLine($"Observation : reponse correcte\n");
Console.WriteLine($"P(C1) = {mDINA.Infer<Bernoulli>(aC1).GetProbTrue():F3}");
Console.WriteLine($"P(C2) = {mDINA.Infer<Bernoulli>(aC2).GetProbTrue():F3}");
Console.WriteLine($"P(C1 ET C2) = {mDINA.Infer<Bernoulli>(toutesCompQ4).GetProbTrue():F3}");
Console.WriteLine($"\nSlip estime : {mDINA.Infer<Beta>(slipQ4).GetMean():F3}");
Console.WriteLine($"Guess estime : {mDINA.Infer<Beta>(guessQ4).GetMean():F3}");

## 5. Relations Many-to-Many

### Probleme

Dans le modele DINA, une question peut necessiter **plusieurs competences**, et une competence peut etre evaluee par **plusieurs questions**.

### Representation avec Subarray

Infer.NET permet d'utiliser `Variable.Subarray` pour extraire les competences requises pour chaque question.

In [ ]:
// Demonstration avec plusieurs etudiants et questions

int nE = 4;
int nC = 3;

// Prior : chaque etudiant a 50% de chance d'avoir chaque competence
Range rE = new Range(nE);
Range rC = new Range(nC);

VariableArray2D<bool> competences = Variable.Array<bool>(rE, rC);
competences[rE, rC] = Variable.Bernoulli(0.5).ForEach(rE, rC);

// Simuler des observations sur plusieurs questions
// Q1 necessite C1 seulement
// Q2 necessite C1 ET C2

// Observations pour Q1 (C1 seulement)
bool[] obsQ1 = { true, true, false, true };  // E1, E2, E4 ont repondu correctement

for (int e = 0; e < nE; e++)
{
    Variable<bool> toutQ1 = competences[e, 0];  // Seulement C1
    Variable<bool> repQ1 = Variable.New<bool>();
    using (Variable.If(toutQ1))
    {
        repQ1.SetTo(Variable.Bernoulli(0.9));  // 1 - slip
    }
    using (Variable.IfNot(toutQ1))
    {
        repQ1.SetTo(Variable.Bernoulli(0.1));  // guess
    }
    repQ1.ObservedValue = obsQ1[e];
}

// Observations pour Q2 (C1 ET C2)
bool[] obsQ2 = { true, false, false, true };

for (int e = 0; e < nE; e++)
{
    Variable<bool> toutQ2 = competences[e, 0] & competences[e, 1];
    Variable<bool> repQ2 = Variable.New<bool>();
    using (Variable.If(toutQ2))
    {
        repQ2.SetTo(Variable.Bernoulli(0.9));
    }
    using (Variable.IfNot(toutQ2))
    {
        repQ2.SetTo(Variable.Bernoulli(0.1));
    }
    repQ2.ObservedValue = obsQ2[e];
}

InferenceEngine mMany = new InferenceEngine();
mMany.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Bernoulli[,] compPost = mMany.Infer<Bernoulli[,]>(competences);

Console.WriteLine("=== Inference des competences (many-to-many) ===");
Console.WriteLine("Q1 necessite C1, Q2 necessite C1 ET C2\n");
Console.WriteLine("Observations : Q1=[T,T,F,T], Q2=[T,F,F,T]\n");

for (int e = 0; e < nE; e++)
{
    Console.Write($"Etudiant {e+1} : ");
    for (int c = 0; c < nC; c++)
    {
        Console.Write($"C{c+1}={compPost[e,c].GetProbTrue():F2} ");
    }
    Console.WriteLine();
}

## 6. Estimation des Parametres Slip/Guess

### Objectif

Apprendre les parametres slip et guess a partir des donnees.

### Approche

- Utiliser des priors Beta sur slip et guess
- L'inference met a jour ces distributions

In [ ]:
// Estimation de slip/guess avec donnees multiples

// Simuler des donnees ou on connait les vraies competences
int nObs = 20;
Random rng = new Random(42);

double vraiSlip = 0.1;
double vraiGuess = 0.2;

// Generer des donnees
bool[] vraiComp = new bool[nObs];    // Vrai etat de competence
bool[] obsRep = new bool[nObs];       // Reponse observee

for (int i = 0; i < nObs; i++)
{
    vraiComp[i] = rng.NextDouble() < 0.6;  // 60% ont la competence
    if (vraiComp[i])
    {
        obsRep[i] = rng.NextDouble() > vraiSlip;  // Correct sauf slip
    }
    else
    {
        obsRep[i] = rng.NextDouble() < vraiGuess;  // Incorrect sauf guess
    }
}

// Modele pour estimer slip et guess
Variable<double> slipEst = Variable.Beta(1, 1).Named("slip");
Variable<double> guessEst = Variable.Beta(1, 1).Named("guess");
Variable<double> pComp = Variable.Beta(1, 1).Named("pComp");

Range rObs = new Range(nObs);
VariableArray<bool> comp = Variable.Array<bool>(rObs);
VariableArray<bool> rep = Variable.Array<bool>(rObs);

comp[rObs] = Variable.Bernoulli(pComp).ForEach(rObs);

using (Variable.ForEach(rObs))
{
    using (Variable.If(comp[rObs]))
    {
        rep[rObs] = !Variable.Bernoulli(slipEst);
    }
    using (Variable.IfNot(comp[rObs]))
    {
        rep[rObs] = Variable.Bernoulli(guessEst);
    }
}

rep.ObservedValue = obsRep;

InferenceEngine mSlipGuess = new InferenceEngine(new ExpectationPropagation());
mSlipGuess.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Estimation Slip/Guess ===");
Console.WriteLine($"Vraies valeurs : slip={vraiSlip}, guess={vraiGuess}\n");
Console.WriteLine($"Estimations :");
Console.WriteLine($"  Slip : {mSlipGuess.Infer<Beta>(slipEst)}");
Console.WriteLine($"  Guess : {mSlipGuess.Infer<Beta>(guessEst)}");
Console.WriteLine($"  P(competence) : {mSlipGuess.Infer<Beta>(pComp)}");

## 7. Comparaison IRT vs DINA

| Aspect | IRT | DINA |
|--------|-----|------|
| **Capacite** | Continue, unidimensionnelle | Discrete, multidimensionnelle |
| **Interpretation** | "Niveau global" | "Competences specifiques" |
| **Complexite** | Simple, robuste | Plus complexe, informatif |
| **Utilisation** | Tests standardises | Diagnostic pedagogique |
| **Inference** | Plus facile | Necessite matrice Q |

## 8. Exercice : Evaluer un Nouvel Etudiant

### Enonce

Un nouvel etudiant passe un test de 5 questions. Le test evalue 2 competences :
- Q1, Q2 : Competence 1 seulement
- Q3, Q4 : Competence 2 seulement
- Q5 : Competences 1 ET 2

Resultats : Q1=correct, Q2=correct, Q3=incorrect, Q4=correct, Q5=incorrect

**Question** : Quelles sont les probabilites que l'etudiant possede C1 et C2 ?

In [ ]:
// EXERCICE : Evaluation d'un nouvel etudiant

Variable<bool> c1 = Variable.Bernoulli(0.5).Named("C1");
Variable<bool> c2 = Variable.Bernoulli(0.5).Named("C2");

double slip = 0.1;
double guess = 0.15;

// Q1 : C1 seulement -> correct
Variable<bool> rQ1 = Variable.New<bool>();
using (Variable.If(c1)) { rQ1.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1)) { rQ1.SetTo(Variable.Bernoulli(guess)); }
rQ1.ObservedValue = true;

// Q2 : C1 seulement -> correct
Variable<bool> rQ2 = Variable.New<bool>();
using (Variable.If(c1)) { rQ2.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1)) { rQ2.SetTo(Variable.Bernoulli(guess)); }
rQ2.ObservedValue = true;

// Q3 : C2 seulement -> incorrect
Variable<bool> rQ3 = Variable.New<bool>();
using (Variable.If(c2)) { rQ3.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c2)) { rQ3.SetTo(Variable.Bernoulli(guess)); }
rQ3.ObservedValue = false;

// Q4 : C2 seulement -> correct
Variable<bool> rQ4 = Variable.New<bool>();
using (Variable.If(c2)) { rQ4.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c2)) { rQ4.SetTo(Variable.Bernoulli(guess)); }
rQ4.ObservedValue = true;

// Q5 : C1 ET C2 -> incorrect
Variable<bool> c1etc2 = c1 & c2;
Variable<bool> rQ5 = Variable.New<bool>();
using (Variable.If(c1etc2)) { rQ5.SetTo(Variable.Bernoulli(1 - slip)); }
using (Variable.IfNot(c1etc2)) { rQ5.SetTo(Variable.Bernoulli(guess)); }
rQ5.ObservedValue = false;

InferenceEngine mEx = new InferenceEngine();
mEx.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Evaluation du nouvel etudiant ===");
Console.WriteLine("Resultats : Q1=T, Q2=T, Q3=F, Q4=T, Q5=F\n");
Console.WriteLine($"P(C1) = {mEx.Infer<Bernoulli>(c1).GetProbTrue():F3}");
Console.WriteLine($"P(C2) = {mEx.Infer<Bernoulli>(c2).GetProbTrue():F3}");
Console.WriteLine($"\nInterpretation :");
Console.WriteLine("- L'etudiant a probablement C1 (2/2 correct sur questions C1)");
Console.WriteLine("- C2 est moins certain (1/2 correct, et Q5 echoue)");

## 9. Resume

| Concept | Description |
|---------|-------------|
| **IRT** | Modele capacite-difficulte pour tests unidimensionnels |
| **DINA** | Modele a competences discretes multiples |
| **Matrice Q** | Definition des competences requises par question |
| **Slip** | Erreur d'inattention (correct -> incorrect) |
| **Guess** | Reponse au hasard (incorrect -> correct) |
| **Many-to-many** | Questions multiples, competences multiples |

---

## Prochaine etape

Dans [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb), nous explorerons :

- Le systeme de classement TrueSkill (Xbox Live)
- L'apprentissage en ligne des competences
- La gestion des matchs nuls
- La visualisation du message passing